In [ ]:
!pip install hazm
!pip install stopwords_guilannlp

In [ ]:
import pandas as pd
import numpy as np
import hazm as hz
from finglish import f2p
import itertools
import re
import pickle
import os
import torch
import gensim
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [3]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [12]:
data = pd.read_csv('Sentiment Analysis.csv' , on_bad_lines='skip' , delimiter='\t')
data.head()

,Unnamed: 0,comment,label,label_id
0,NaN,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1.0
1,NaN,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0.0
2,NaN,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1.0
3,NaN,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0.0
4,NaN,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0.0


In [24]:
data = data[['comment' , 'label' , 'label_id']]
data.dropna(inplace=True)

<ipython-input-24-4d6c008ed5f0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [25]:
data.query('label == "HAPPY" and label_id==1.0')

,comment,label,label_id


In [26]:
data['label_id'] = data['label_id'].astype(int)
data.head()

,comment,label,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [34]:
def fixup(x):
    x = x.replace('\u200c', '').replace('\xa0','').replace('\r\n',' ').replace('|',' ')
    return x

normalizer = hz.Normalizer()

def my_tokenizer(text):
  text = re.sub(r"[\{\}\؛\*\=\-\+\/\n]"," ",str(text))
  text = re.sub("[ ]+"," ",text)
  text = re.sub("\!+","!",text)
  text = re.sub("[؟]+","؟",text)
  text = re.sub("\?+","?",text)
  text = re.sub("[.]+","",text)
  text = re.sub("[،]+","",text)
  if(bool(re.match('^[a-zA-Z]',text))==True):
    text=f2p(text)
  for c in "..آابپتثجچحخدذرزژسشصضطظعغفقکگلمنوهیئ":
    text = re.sub(f"[{c}]+", c, text)
  text = fixup(normalizer.normalize(text))
  words = []
  words.append(hz.word_tokenize(text))
  return words

nested_train_data = data['comment'].apply(my_tokenizer)
data['comment'] = list(itertools.chain(*nested_train_data))

In [35]:
data

,comment,label,label_id
0,"[واقعا, حیف, وقت, که, بنویسم, سرویس, دهیتون, ش...",SAD,1
1,"[قرار, بود, ۱, ساعته, برسه, ولی, نیم, ساعت, زو...",HAPPY,0
2,"[قیمت, این, مدل, اصلا, با, کیفیتش, سازگاری, ند...",SAD,1
3,"[عالی, بود, همه, چه, درست, و, به, اندازه, و, ک...",HAPPY,0
4,"[شیرینی, وانیلی, فقط, یک, مدل, بود]",HAPPY,0
...,...,...,...
69995,"[سلام, من, به, فاکتور, غذاهای, که, سفارش, میدم...",SAD,1
69996,"[سایز, پیتزا, نسبت, به, سفارشاتی, که, قبلا, گذ...",SAD,1
69997,"[من, قارچ, اضافه, رو, اضافه, کرده_بودم, بودم, ...",HAPPY,0
69998,"[همرو, بعد, ۲ساعت, تاخیر, اشتباه, آوردن, پولشم...",SAD,1


In [36]:
words_count = data['comment'].apply(len)

print('Min length =', words_count.min())
print('Max length =', words_count.max())
print('Mean = {:.2f}'.format(words_count.mean()))
print('Std  = {:.2f}'.format(words_count.std()))
print('mean + 2 * sigma = {:.2f}'.format(words_count.mean() + 2.0 * words_count.std()))

Min length = 2
Max length = 373
Mean = 18.53
Std  = 15.79
mean + 2 * sigma = 50.11


In [37]:
max_len = 32
PAD = ''

def padding_and_trimming(tokens):
  if len(tokens) < max_len:
      num_pads = max_len - len(tokens)
      tokens = [PAD] * num_pads + tokens
  elif len(tokens) > max_len:
      tokens = tokens[:max_len]
  return tokens

In [38]:
data['comment'] = data['comment'].apply(padding_and_trimming)

In [ ]:
!wget 'http://vectors.nlpl.eu/repository/20/61.zip' -O './w2vec.zip'
!unzip ./w2vec.zip -d ./w2vec

w2v_model = gensim.models.KeyedVectors.load_word2vec_format('./w2vec/model.txt', binary=False, unicode_errors='replace')
w2v_weights = torch.FloatTensor(w2v_model.vectors)

In [40]:
class LSTMClassifier(nn.Module):
  def __init__(self, hidden_size, batch_size, layers_count):
    super(LSTMClassifier, self).__init__()
    self.hidden_size = hidden_size
    self.batch_size = batch_size
    self.layers_count = layers_count

    self.embedding = nn.Embedding.from_pretrained(w2v_weights)
    self.lstm = nn.LSTM(100, hidden_size, layers_count, bidirectional=True, batch_first=True)
    self.classifier_layer = nn.Sequential(
        nn.Linear(2*hidden_size, 100),
        nn.ReLU(),
        nn.Linear(100, 2)
    )
    self.hidden = self.init_hidden()

  def init_hidden(self):
    h = torch.autograd.Variable(torch.zeros((2*self.layers_count, self.batch_size, self.hidden_size)).to(device))
    c = torch.autograd.Variable(torch.zeros((2*self.layers_count, self.batch_size, self.hidden_size)).to(device))
    return h, c

  def forward(self, x):
    x = self.embedding(x)
    x, self.hidden = self.lstm(x, self.hidden)
    x = x.permute(1, 0, 2).detach()
    x = self.classifier_layer(x[-1])
    return x

In [ ]:
BATCH_SIZE = 128
lstm_model = LSTMClassifier(hidden_size=512, batch_size=BATCH_SIZE, layers_count=1)

if gpu_enable:
  lstm_model = lstm_model.cuda()
criterion = nn.CrossEntropyLoss()
if gpu_enable:
  criterion = criterion.cuda()

optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.01, betas=(0.7, 0.99))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.975)

In [47]:
PAD = ''
UNK = ''

class TDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.X_train = dataset['comment']
        self.y_train = dataset['label_id']

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, index):
        vectors = []
        for token in self.X_train[index]:
          if token == PAD:
            vectors.append(1)
            continue
          try:
            vectors.append(w2v_model.vocab[token].index)
          except KeyError:
            vectors.append(2)
        return torch.tensor(vectors), torch.tensor(self.y_train[index])


dataset = TDataset(data)

In [ ]:
train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

lstm_model.train()

losses = []
for epoch in range(5):
  total_loss = 0
  for i, (inputs, targets) in enumerate(train_dataloader):
    optimizer.zero_grad()

    inputs = inputs.to(device)
    targets = targets.to(device)
    outputs = lstm_model(inputs)
    
    loss = criterion(outputs, targets)
    loss.backward()
    scheduler.step()
    total_loss += loss.item()

    print(f'Epoch {epoch + 1}/5 : step {i + 1}/{len(dataset) // BATCH_SIZE}, loss: {loss.item()}')